## Cyberpunks. Urbit Owners

In [1]:
from IPython.core.display import display, HTML

from config import ERC721_ANALYSIS_DATASET_NAME, AZIMUTH_POINTS_TABLE_NAME, SNAPSHOT_TS
from src.utils_bigquery import get_df, create_table_from_df
from src.extractor_urbit import extract_point_data

SELECT_TRANSFERRED_TOKENS = True
DROP_TABLES = True
CREATE_TABLES = True

AZIMUTH_CONTRACT_ADDRESS = '0x223c067F8CF28ae173EE5CafEa60cA44C335fecB'.lower()
LINEAR_STAR_RELEASE_CONTRACT_ADDRESS= '0x86cd9cd0992F04231751E3761De45cEceA5d1801'.lower()
SPAWNED_EVENT_TOPIC = '0xb2d3a6e7a339f5c8ff96265e2f03a010a8541070f3744a247090964415081546'
ZERO_ADDRESS = '0x0000000000000000000000000000000000000000'
AZIMUTH_CONTRACT_CREATE_TS = '2018-11-27 00:00:00'

### Get List of Transferred Tokens

In [2]:
if SELECT_TRANSFERRED_TOKENS:
    query_1 = f'''
    SELECT cast(topics[ORDINAL(3)] AS INT64) as point
    FROM `bigquery-public-data.crypto_ethereum.logs`
    WHERE block_timestamp > '{AZIMUTH_CONTRACT_CREATE_TS}'
      AND block_timestamp < '{SNAPSHOT_TS}'
      AND address = '{AZIMUTH_CONTRACT_ADDRESS}'
      AND topics[ORDINAL(1)] = '{SPAWNED_EVENT_TOPIC}'
    '''

    azimuth_tokens_list = list(get_df(query=query_1).point)
    print(f'Number of Azimuth Tokens: {len(azimuth_tokens_list):>,}')

Number of Azimuth Tokens: 75,704


### Get Point Owners and Types

In [3]:
points_df = extract_point_data(point_list=azimuth_tokens_list, pool_size=10, chunk_size=128)

Processing 75704 points split into 592 chunks on 10 processes:
Points 42952:19765 extracted
Points 35487:3992 extracted
Points 42383:46901 extracted
Points 57330:42624 extracted
Points 1793055392:54359 extracted
Points 63500:9228 extracted
Points 51554:40149 extracted
Points 2344290486:2335 extracted
Points 35940:23831 extracted
Points 62724:30467 extracted
Points 47157:25141 extracted
Points 42880:28431 extracted
Points 54615:59990 extracted
Points 20021:59999 extracted
Points 9484:24844 extracted
Points 4248:20024 extracted
Points 2591:20474 extracted
Points 24087:40208 extracted
Points 40405:13317 extracted
Points 30723:2934076061 extracted
Points 25397:30478 extracted
Points 28687:55953 extracted
Points 39511:63062 extracted
Points 60255:36418 extracted
Points 40981:23830 extracted
Points 5374720:55136 extracted
Points 20280:535 extracted
Points 22266:36623 extracted
Points 13573:41826 extracted
Points 3790012834:3924621496 extracted
Points 27918:1593 extracted
Points 36674:40505 e

In [4]:
display(HTML(points_df.head().to_html(index=False, notebook=True, show_dimensions=False)))

point,point_size,point_type,owner,parent_point
2344290486,2,planet,0xff4304024e87b2787e4b3363701111c1c6efb5b6,2230
1972597405,2,planet,0xff4304024e87b2787e4b3363701111c1c6efb5b6,29341
1236137653,2,planet,0xff4304024e87b2787e4b3363701111c1c6efb5b6,63157
410138016,2,planet,0xff4304024e87b2787e4b3363701111c1c6efb5b6,13728
1467395254,2,planet,0xfc2d1d75b6b6f81c609f22ee386bc69667a92ac7,44214


### Insert Point Data

In [5]:
create_table_from_df(source_df=points_df,
                     dataset_name=ERC721_ANALYSIS_DATASET_NAME,
                     table_name=AZIMUTH_POINTS_TABLE_NAME,
                     drop_existing_table=DROP_TABLES)

Table erc721_analysis:azimuth_points has been deleted.
Table erc721_analysis:azimuth_points has been created.


True